In [10]:
import argparse
import pandas as pd
import numpy as np
from numpy import savez_compressed, load
import re
import time
import os
import pickle
from tqdm import tqdm
tqdm.pandas(position=0,leave=True)
from collections import Counter
import textwrap

In [2]:
my_folder="s3://trident-retention-output/"
folder = 's3://trident-retention-data/askunum/'

start=time.time()
askunum_text=pd.read_pickle(os.path.join(my_folder,'askunum_text_pickle'))
end=time.time()
print("It took {:0.4f} seconds to read text data".format(end-start))

churn_data=pd.read_pickle(os.path.join(my_folder,'churn_data_pickle'))

It took 194.4411 seconds to read text data


In [11]:
askunum_text.head(2)

,Id,ParentId,Incoming,MessageDate,TextBody,CreatedDate,ClosedDate,Subtype,year,month,account_id,unum_id
1136587,02s3x00001lwyxeAAA,5003x00002CFUfAAAX,False,2021-10-25T17:07:44.000+0000,re 436760 fo-pittsburgh pa- pam schon phone- 4...,2021-10-22 17:18:20+00:00,2021-10-25T17:08:45.000+0000,Charge or Credit Clarification,2021,10,0013x00002Ipz9G,1000110382
1136585,02s3x00001lwzYbAAI,5003x00002CFUfAAAX,False,2021-10-25T17:08:22.000+0000,"re 436760 hi pam, i hope this email finds you ...",2021-10-22 17:18:20+00:00,2021-10-25T17:08:45.000+0000,Charge or Credit Clarification,2021,10,0013x00002Ipz9G,1000110382


In [4]:
askunum_text.unum_id.unique().shape[0], askunum_text.shape[0]

(89275, 5016545)

In [5]:
churn_data=churn_data.set_index(["unum_id","end_date"])
churn_data.head(2)

,,policy_id,pivot_date,year,month,start_date,churn
unum_id,end_date,,,,,,
GRA00956433,2021-05-01,956433,2021-08-01,2021,8,2020-11-01,1
GRA00956388,2021-03-01,956388,2021-06-01,2021,6,2020-09-01,1


In [8]:
file_name="unique_unum_v1"
my_folder="s3://trident-retention-output/"
unique_unum_id=pd.read_csv(os.path.join(my_folder,file_name+".csv"), usecols=["unum_id"])["unum_id"].values.squeeze()

churn_df=churn_data[churn_data.index.get_level_values("unum_id").isin(unique_unum_id.astype(str))]
text_df=askunum_text[askunum_text["unum_id"].isin(unique_unum_id.astype(str))]
print(churn_df.shape)
print(text_df.shape)

(33362, 6)
(491303, 12)


In [22]:
def dataframe_to_dictionary(df, key_column): #cost 10 min
    start = time.time()
    dict_of_dataframes = dict()
    df["year_month"]=pd.to_datetime(df.apply(lambda x: str(x['year'])+'-' + str(x['month']) ,axis=1),format="%Y-%m")
      
    counter = 0 
    for id, group in tqdm(df.groupby(key_column),total=df.unum_id.unique().shape[0], position=0, leave=True):
        group = group.sort_values("year_month")
        group = group.set_index("year_month")
        dict_of_dataframes[id] = group
        counter += 1
    return dict_of_dataframes
df_dict = dataframe_to_dictionary(text_df, 'unum_id')

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
100%|██████████| 8927/8927 [00:05<00:00, 1717.61it/s]


In [18]:
churn_data=pd.read_pickle(os.path.join(my_folder,'churn_data_pickle'))

'1000110382'

In [29]:
df_dict[list(df_dict.keys())[0]].head()

,Id,ParentId,Incoming,MessageDate,TextBody,CreatedDate,ClosedDate,Subtype,year,month,account_id,unum_id
year_month,,,,,,,,,,,,
2021-10-01,02s3x00001lwyxeAAA,5003x00002CFUfAAAX,False,2021-10-25T17:07:44.000+0000,re 436760 fo-pittsburgh pa- pam schon phone- 4...,2021-10-22 17:18:20+00:00,2021-10-25T17:08:45.000+0000,Charge or Credit Clarification,2021,10,0013x00002Ipz9G,1000110382
2021-10-01,02s3x00001lwzYbAAI,5003x00002CFUfAAAX,False,2021-10-25T17:08:22.000+0000,"re 436760 hi pam, i hope this email finds you ...",2021-10-22 17:18:20+00:00,2021-10-25T17:08:45.000+0000,Charge or Credit Clarification,2021,10,0013x00002Ipz9G,1000110382
2021-10-01,02s3x00001mVWzdAAG,5003x00002CFUfAAAX,True,2021-11-17T18:09:06.000+0000,. . we still have not been contacted by anyone...,2021-10-22 17:18:20+00:00,2021-10-25T17:08:45.000+0000,Charge or Credit Clarification,2021,10,0013x00002Ipz9G,1000110382
2021-11-01,02s3x00001mVWzeAAG,5003x00002D70pBAAR,True,2021-11-17T18:09:06.000+0000,. . we still have not been contacted by anyone...,2021-11-17 18:11:50+00:00,2021-11-17T22:27:09.000+0000,Charge or Credit Clarification,2021,11,0013x00002Ipz9G,1000110382
2021-11-01,02s3x00001mVcioAAC,5003x00002D70pBAAR,False,2021-11-17T22:23:50.000+0000,"re 436760 good afternoon pam, i do apologize y...",2021-11-17 18:11:50+00:00,2021-11-17T22:27:09.000+0000,Charge or Credit Clarification,2021,11,0013x00002Ipz9G,1000110382


In [ ]:
df_dict[list(df_dict.keys())[0]]

In [39]:
def add_dictionary_of_dataframes(churn_data, dict_of_df,  num_months_to_use=6): 
    
    unum_id=[]
    policy_id=[]
    pivot_date=[]
    year=[]
    month=[]
    start_date=[]
    end_date=[]
    Full_TextBody=[]
    Client_TextBody=[]
    Latest_TextBody=[]
    EMAIL_COUNT=[]
    ISSUE_COUNT=[]
    DURATION=[]
    SUBTYPE=[]
    churn=[]
    
    for index,row in tqdm(churn_data.iterrows(), total=churn_data.shape[0]):
        id=row["unum_id"]
        last_date=row["end_date"]
        if id not in dict_of_df:
            continue
        data = dict_of_df[id]
        try:
            end_index = data.index.get_loc(last_date)
            tempt = data.iloc[end_index+1-num_months_to_use:end_index+1]
        except:
            continue
        
        if tempt.empty:
            continue
            
        tempt=tempt.reset_index()
        tempt.dropna(subset=['TextBody'],inplace=True)
        tempt.sort_values(["unum_id","MessageDate"],inplace=True,ascending=True)
        
        subtype=Counter(tempt[tempt["Incoming"]==True].Subtype.values)
        email_counts = tempt[tempt["Incoming"]==True].shape[0]
        issue_counts = tempt["ParentId"].unique().shape[0]
        tempt['askunum_days'] = (pd.to_datetime(tempt['ClosedDate']) - pd.to_datetime(tempt['CreatedDate'])).apply(lambda x: (x.days * 24 + x.seconds / 3600)/24)
        tempt2=tempt.drop_duplicates(subset=["ParentId"])
        duration=np.sum(tempt2['askunum_days'].values)

        tempt_1=tempt.groupby(["unum_id"])["TextBody"].apply(lambda x : ".".join(x)).reset_index()
        Full_TextBody.append(tempt_1["TextBody"][0])

        tempt_2=tempt[tempt["Incoming"]==True]
        tempt_2=tempt_2.groupby(["unum_id"])["TextBody"].apply(lambda x : ".".join(x)).reset_index()
        if tempt_2.empty:
            Client_TextBody.append(None)
        else:
            Client_TextBody.append(tempt_2["TextBody"][0])

        tempt_3=tempt.sort_values(["unum_id","ParentId","MessageDate"],ascending=True)
        tempt_3.drop_duplicates(subset=["unum_id","ParentId"],keep="last",inplace=True)
        tempt_3=tempt_3.groupby(["unum_id"])["TextBody"].apply(lambda x : ".".join(x)).reset_index()
        if tempt_3.empty:
            Latest_TextBody.append(None)
        else:
            Latest_TextBody.append(tempt_3["TextBody"][0])        

        EMAIL_COUNT.append(email_counts)
        ISSUE_COUNT.append(issue_counts)
        DURATION.append(duration)
        SUBTYPE.append(subtype)
    
        unum_id.append(row["unum_id"])
        policy_id.append(row["policy_id"])
        pivot_date.append(row["pivot_date"])
        year.append(row["year"])
        month.append(row["month"])
        start_date.append(row["start_date"])
        end_date.append(row["end_date"])
        churn.append(row["churn"])
        
    churn_text_data=pd.DataFrame({"unum_id":unum_id,"policy_id":policy_id,"pivot_date":pivot_date,"year":year,"month":month,\
                                 "start_date":start_date,"end_date":end_date,"Full_TextBody":Full_TextBody,"Client_TextBody":Client_TextBody,\
                                  "Latest_TextBody":Latest_TextBody,"email_counts":EMAIL_COUNT,"issue_counts":ISSUE_COUNT,"duration":DURATION,"subtype":SUBTYPE,"churn":churn})
    
    ## drop data if Client email is None
    churn_text_data.dropna(subset=["Client_TextBody"],inplace=True)
    
    # churn_text_data.drop_duplicates(inplace=True)
    churn_text_data.sort_values(by=["unum_id","policy_id","year","month"],ascending=False,inplace=True)
    churn_text_data['unum_id']=churn_text_data['unum_id'].apply(str)
    churn_text_data['policy_id']=churn_text_data['policy_id'].apply(int)
    churn_text_data=churn_text_data.reset_index(drop=True)
    
    # churn_text_data.to_pickle(os.path.join(my_folder,args.output_name))
    
    return churn_text_data
    
%pdb
df=add_dictionary_of_dataframes(churn_data, df_dict, num_months_to_use=6)

Automatic pdb calling has been turned OFF


100%|██████████| 445559/445559 [00:29<00:00, 15259.96it/s]


In [40]:
df.head()

,unum_id,policy_id,pivot_date,year,month,start_date,end_date,Full_TextBody,Client_TextBody,Latest_TextBody,email_counts,issue_counts,duration,subtype,churn
0,160806244,416180,2021-12-01,2021,12,2021-03-01,2021-09-01,hi ask unum please escalate to our ltc unit an...,hi ask unum please escalate to our ltc unit an...,. . please look at my original question below....,3,2,0.018032,{'Employee Coding': 3},0
1,160760087,600494,2020-01-01,2020,1,2019-04-01,2019-10-01,name lindsey roberson phone 770-479-7777 respo...,name lindsey roberson phone 770-479-7777 respo...,"re 600494 hi lindsey, both the basic life and ...",2,4,4.045498,"{'Portability or Conversion': 1, 'Add, Remove,...",0
2,160145551,403472,2020-12-01,2020,12,2020-03-01,2020-09-01,"re715789 hello veronica, we have completed thi...",". . good day, i am having trouble logging in o...","re715789 hello veronica, we have completed thi...",2,4,2.949398,"{'Password Reset or Login issues': 1, 'Dental/...",0
3,160145551,403471,2020-12-01,2020,12,2020-03-01,2020-09-01,"re715789 hello veronica, we have completed thi...",". . good day, i am having trouble logging in o...","re715789 hello veronica, we have completed thi...",2,4,2.949398,"{'Password Reset or Login issues': 1, 'Dental/...",0
4,160113964,600388,2020-01-01,2020,1,2019-04-01,2019-10-01,"re 543722 hello reyne, . i have ordered the re...",". . thank you so very much rey reyne j. boik, ...",". . thank you so very much rey reyne j. boik, ...",4,3,3.760451,{'Employee Assistance Plan (EAP) Wallet Card':...,0
